###  Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import nltk
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')

### AP

In [ ]:
df_business = pd.read_csv('/content/drive/MyDrive/web_scraped_data/AP/apnews_business.csv')

In [ ]:
df_business.head(20)

,headline,article_link,article_title,article_content
0,US and China to start talks over trade war thi...,https://www.bbc.com/news/articles/c8rgrejkvmjo,US and China to start talks over trade war thi...,US and Chinese officials are set to start talk...
1,UK and India agree trade deal after three year...,https://www.bbc.com/news/articles/c5y6y90e5vzo,UK and India agree trade deal after three year...,The UK and India have agreed a trade deal that...
2,NaN,https://cloud.email.bbc.com/WorldofBusiness_Ne...,NaN,NaN
3,NaN,https://www.bbc.com/news/articles/c5y6y90e5vzo,UK and India agree trade deal after three year...,The UK and India have agreed a trade deal that...
4,Analysis: UK-India deal secured as Trump tarif...,https://www.bbc.com/news/articles/cn919g2v3w2o,Faisal Islam: Trump tariffs may have helped dr...,It has long been the great prize for Britain's...
5,EU plans to end Russian gas imports by end of ...,https://www.bbc.com/news/articles/c04547pr9vqo,EU plans to end Russian gas imports by end of ...,"The European Commission has published a ""roadm..."
6,UK's Deliveroo to be bought by US firm DoorDas...,https://www.bbc.com/news/articles/c5yr1nz7jwko,UK's Deliveroo to be bought by US firm DoorDas...,"Deliveroo, the food delivery app, has agreed t..."
7,World of Business,https://cloud.email.bbc.com/WorldofBusiness_Ne...,NaN,NaN
8,Car giant Ford and Barbie maker Mattel warn ov...,https://www.bbc.com/news/articles/cly1e7ll07do,Car giant Ford and Barbie maker Mattel warn ov...,Barbie maker Mattel says it will put up the pr...
9,India worried about Chinese 'dumping' as trade...,https://www.bbc.com/news/articles/ckg5d505v8xo,India worried about Chinese 'dumping' as trade...,The pace at 64-year-old Thirunavkarsu's spinni...


In [8]:
def missing_values_table(df):
    '''Calculating missing values by column'''
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
    mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)

    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
    return mis_val_table_ren_columns


In [ ]:
missing_values_table(df_business)

Your selected dataframe has 4 columns.
There are 3 columns that have missing values.


,Missing Values,% of Total Values
article_content,3,15.0
headline,2,10.0
article_title,2,10.0


In [ ]:
df_business.rename(columns={'article_link': 'article_url'}, inplace=True)

In [ ]:
df_business.head()

,headline,article_url,article_title,article_content
0,US and China to start talks over trade war thi...,https://www.bbc.com/news/articles/c8rgrejkvmjo,US and China to start talks over trade war thi...,US and Chinese officials are set to start talk...
1,UK and India agree trade deal after three year...,https://www.bbc.com/news/articles/c5y6y90e5vzo,UK and India agree trade deal after three year...,The UK and India have agreed a trade deal that...
2,NaN,https://cloud.email.bbc.com/WorldofBusiness_Ne...,NaN,NaN
3,NaN,https://www.bbc.com/news/articles/c5y6y90e5vzo,UK and India agree trade deal after three year...,The UK and India have agreed a trade deal that...
4,Analysis: UK-India deal secured as Trump tarif...,https://www.bbc.com/news/articles/cn919g2v3w2o,Faisal Islam: Trump tariffs may have helped dr...,It has long been the great prize for Britain's...


In [ ]:
df_business = df_business.dropna()

In [ ]:
missing_values_table(df_business)

Your selected dataframe has 4 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [ ]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 0 to 19
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   headline         16 non-null     object
 1   article_url      16 non-null     object
 2   article_title    16 non-null     object
 3   article_content  16 non-null     object
dtypes: object(4)
memory usage: 640.0+ bytes


In [ ]:
# Entertainment
df_entertainment = pd.read_csv('/content/drive/MyDrive/web_scraped_data/AP/apnews_entertainment.csv')

In [ ]:
df_entertainment.head()

,headline,article_url,article_title,article_content,published_date
0,"The Met Gala is over, but dandyism isn’t. Here...",https://apnews.com/article/dandyism-real-life-...,"The Met Gala is over, but dandyism isn’t. Here...",NEW YORK (AP) — Michael Henry Adams had reache...,NaN
1,Dandyism was at the heart of Monday’s Met Gala...,https://apnews.com/article/dandyism-real-life-...,"The Met Gala is over, but dandyism isn’t. Here...",NEW YORK (AP) — Michael Henry Adams had reache...,NaN
2,Smokey Robinson accused by former housekeepers...,https://apnews.com/article/smokey-robinson-la-...,Smokey Robinson accused by former housekeepers...,LOS ANGELES (AP) — Four former housekeepers of...,NaN
3,"In pinstripes and hats, with canes and twists ...",https://apnews.com/article/met-gala-fashion-20...,"In pinstripes and hats, with canes and twists ...",NEW YORK (AP) — Pinstripes. All Ways. Hats gal...,NaN
4,Man arrested after ramming car into front gate...,https://apnews.com/article/jennifer-aniston-lo...,Man arrested after ramming car into front gate...,LOS ANGELES (AP) — A man was arrested after cr...,NaN


In [ ]:
missing_values_table(df_entertainment)

Your selected dataframe has 5 columns.
There are 1 columns that have missing values.


,Missing Values,% of Total Values
published_date,16,100.0


In [ ]:
df_entertainment.drop(columns=['published_date'], inplace=True)


In [ ]:
df_entertainment

,headline,article_url,article_title,article_content
0,"The Met Gala is over, but dandyism isn’t. Here...",https://apnews.com/article/dandyism-real-life-...,"The Met Gala is over, but dandyism isn’t. Here...",NEW YORK (AP) — Michael Henry Adams had reache...
1,Dandyism was at the heart of Monday’s Met Gala...,https://apnews.com/article/dandyism-real-life-...,"The Met Gala is over, but dandyism isn’t. Here...",NEW YORK (AP) — Michael Henry Adams had reache...
2,Smokey Robinson accused by former housekeepers...,https://apnews.com/article/smokey-robinson-la-...,Smokey Robinson accused by former housekeepers...,LOS ANGELES (AP) — Four former housekeepers of...
3,"In pinstripes and hats, with canes and twists ...",https://apnews.com/article/met-gala-fashion-20...,"In pinstripes and hats, with canes and twists ...",NEW YORK (AP) — Pinstripes. All Ways. Hats gal...
4,Man arrested after ramming car into front gate...,https://apnews.com/article/jennifer-aniston-lo...,Man arrested after ramming car into front gate...,LOS ANGELES (AP) — A man was arrested after cr...
5,Things to know about the trial of Sean ‘Diddy’...,https://apnews.com/article/diddy-combs-trial-j...,Things to know about the trial of Sean ‘Diddy’...,NEW YORK (AP) — The sex trafficking trial of S...
6,Summer Movie Guide 2025: Here’s what’s coming ...,https://apnews.com/article/summer-movie-calend...,Summer Movie Guide 2025: Here’s what’s coming ...,Ethan Hunt’s last mission? A new Superman? Hap...
7,James Gunn predicts summer of ‘Superman’ to th...,https://apnews.com/article/warner-bros-superma...,James Gunn predicts summer of ‘Superman’ to th...,LAS VEGAS (AP) — The 2025 box office has been ...
8,Summer Movies: 11 breakout actors to watch,https://apnews.com/article/summer-movies-2025-...,Summer Movies: 11 breakout actors to watch,Many of the big movies this summer might come ...
9,"Movie Review: ‘Thunderbolts’ is Marvel, and Fl...",https://apnews.com/article/thunderbolts-review...,"Movie Review: ‘Thunderbolts’ is Marvel, and Fl...","As they so often do in Marvel Land, worlds col..."


In [ ]:
missing_values_table(df_entertainment)

Your selected dataframe has 4 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [ ]:
df_entertainment.head()

,headline,article_url,article_title,article_content
0,"The Met Gala is over, but dandyism isn’t. Here...",https://apnews.com/article/dandyism-real-life-...,"The Met Gala is over, but dandyism isn’t. Here...",NEW YORK (AP) — Michael Henry Adams had reache...
1,Dandyism was at the heart of Monday’s Met Gala...,https://apnews.com/article/dandyism-real-life-...,"The Met Gala is over, but dandyism isn’t. Here...",NEW YORK (AP) — Michael Henry Adams had reache...
2,Smokey Robinson accused by former housekeepers...,https://apnews.com/article/smokey-robinson-la-...,Smokey Robinson accused by former housekeepers...,LOS ANGELES (AP) — Four former housekeepers of...
3,"In pinstripes and hats, with canes and twists ...",https://apnews.com/article/met-gala-fashion-20...,"In pinstripes and hats, with canes and twists ...",NEW YORK (AP) — Pinstripes. All Ways. Hats gal...
4,Man arrested after ramming car into front gate...,https://apnews.com/article/jennifer-aniston-lo...,Man arrested after ramming car into front gate...,LOS ANGELES (AP) — A man was arrested after cr...


In [ ]:
# Politics
df_politics = pd.read_csv('/content/drive/MyDrive/web_scraped_data/AP/apnews_politics.csv')

In [ ]:
df_politics.head()

,headline,article_url,article_title,article_content,published_date
0,America’s legal system is confusing. Here are ...,https://apnews.com/article/trump-lawsuits-defi...,NaN,NaN,NaN
1,Trump plans to announce the US will call the P...,https://apnews.com/article/trump-persian-gulf-...,NaN,NaN,NaN
2,"Ground Game: Trump team projects calm, Thune’s...",https://apnews.com/newsletter/ground-game/apri...,"Ground Game: Trump team projects calm, Thune’s...",This newsletter was originally sent out via em...,NaN
3,Trump says only 21 hostages held by Hamas in G...,https://apnews.com/article/trump-hamas-gaza-is...,Trump says only 21 hostages held by Hamas in G...,WASHINGTON (AP) — President Donald Trump said ...,NaN
4,New York GOP congressman hit with questions ab...,https://apnews.com/video/new-york-gop-congress...,NaN,NaN,NaN


In [ ]:
missing_values_table(df_politics)

Your selected dataframe has 5 columns.
There are 3 columns that have missing values.


,Missing Values,% of Total Values
published_date,21,100.0
article_title,8,38.1
article_content,8,38.1


In [ ]:
df_politics.drop(columns=['published_date'], inplace=True)

In [ ]:
df_politics.head()

,headline,article_url,article_title,article_content
0,America’s legal system is confusing. Here are ...,https://apnews.com/article/trump-lawsuits-defi...,NaN,NaN
1,Trump plans to announce the US will call the P...,https://apnews.com/article/trump-persian-gulf-...,NaN,NaN
2,"Ground Game: Trump team projects calm, Thune’s...",https://apnews.com/newsletter/ground-game/apri...,"Ground Game: Trump team projects calm, Thune’s...",This newsletter was originally sent out via em...
3,Trump says only 21 hostages held by Hamas in G...,https://apnews.com/article/trump-hamas-gaza-is...,Trump says only 21 hostages held by Hamas in G...,WASHINGTON (AP) — President Donald Trump said ...
4,New York GOP congressman hit with questions ab...,https://apnews.com/video/new-york-gop-congress...,NaN,NaN


In [ ]:
missing_values_table(df_politics)

Your selected dataframe has 4 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values
article_title,8,38.1
article_content,8,38.1


In [ ]:
df_politics

,headline,article_url,article_title,article_content
0,America’s legal system is confusing. Here are ...,https://apnews.com/article/trump-lawsuits-defi...,NaN,NaN
1,Trump plans to announce the US will call the P...,https://apnews.com/article/trump-persian-gulf-...,NaN,NaN
2,"Ground Game: Trump team projects calm, Thune’s...",https://apnews.com/newsletter/ground-game/apri...,"Ground Game: Trump team projects calm, Thune’s...",This newsletter was originally sent out via em...
3,Trump says only 21 hostages held by Hamas in G...,https://apnews.com/article/trump-hamas-gaza-is...,Trump says only 21 hostages held by Hamas in G...,WASHINGTON (AP) — President Donald Trump said ...
4,New York GOP congressman hit with questions ab...,https://apnews.com/video/new-york-gop-congress...,NaN,NaN
5,Trump says he wants to reopen the Alcatraz pri...,https://apnews.com/video/trump-says-he-wants-t...,NaN,NaN
6,Trump does impression of transgender weightlif...,https://apnews.com/video/trump-does-impression...,NaN,NaN
7,Core Democratic groups are preparing to be tar...,https://apnews.com/article/democrats-trump-act...,NaN,NaN
8,"Even in Alabama, some Republicans worry about ...",https://apnews.com/article/trump-alabama-repub...,NaN,NaN
9,Where are federal jobs affected by DOGE cuts? ...,https://apnews.com/article/doge-federal-job-cu...,NaN,NaN


In [ ]:
df_politics['article_title'] = df_politics['article_title'].fillna(df_politics['headline'])


In [ ]:
df_politics['article_content'] = df_politics['article_content'].fillna(df_politics['headline'])


In [ ]:
df_politics.head()

,headline,article_url,article_title,article_content
0,America’s legal system is confusing. Here are ...,https://apnews.com/article/trump-lawsuits-defi...,America’s legal system is confusing. Here are ...,America’s legal system is confusing. Here are ...
1,Trump plans to announce the US will call the P...,https://apnews.com/article/trump-persian-gulf-...,Trump plans to announce the US will call the P...,Trump plans to announce the US will call the P...
2,"Ground Game: Trump team projects calm, Thune’s...",https://apnews.com/newsletter/ground-game/apri...,"Ground Game: Trump team projects calm, Thune’s...",This newsletter was originally sent out via em...
3,Trump says only 21 hostages held by Hamas in G...,https://apnews.com/article/trump-hamas-gaza-is...,Trump says only 21 hostages held by Hamas in G...,WASHINGTON (AP) — President Donald Trump said ...
4,New York GOP congressman hit with questions ab...,https://apnews.com/video/new-york-gop-congress...,New York GOP congressman hit with questions ab...,New York GOP congressman hit with questions ab...


In [ ]:
missing_values_table(df_politics)

Your selected dataframe has 4 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [ ]:
# Sports
df_sports= pd.read_csv('/content/drive/MyDrive/web_scraped_data/AP/apnews_sports.csv')

In [ ]:
df_sports.head()

,headline,article_url,article_title,article_content,published_date
0,Curry leaves Warriors’ win vs. Wolves with ham...,https://apnews.com/article/nba-playoffs-warrio...,NaN,NaN,NaN
1,Golden State star Stephen Curry was sidelined ...,https://apnews.com/article/nba-playoffs-warrio...,Curry leaves Warriors’ win vs. Wolves with ham...,MINNEAPOLIS (AP) — Golden State star Stephen C...,NaN
2,Haliburton’s 3 with 1.1 seconds left gives Pac...,https://apnews.com/article/nba-playoffs-pacers...,Haliburton’s 3 with 1.1 seconds left gives Pac...,CLEVELAND (AP) — Tyrese Haliburton said he kno...,NaN
3,Jaccob Slavin scores in OT as the Hurricanes b...,https://apnews.com/article/hurricanes-capitals...,Jaccob Slavin scores in OT as the Hurricanes b...,WASHINGTON (AP) — Very few people in the arena...,NaN
4,Oilers rally for NHL-record fifth straight tim...,https://apnews.com/article/oilers-golden-knigh...,Oilers rally for NHL-record fifth straight tim...,LAS VEGAS (AP) — One thing is becoming clear d...,NaN


In [ ]:
df_sports.drop(columns=['published_date'], inplace=True)

In [ ]:
missing_values_table(df_sports)

Your selected dataframe has 4 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values
article_title,4,16.0
article_content,4,16.0


In [ ]:
df_sports['article_content'] = df_sports['article_content'].fillna(df_sports['headline'])
df_sports['article_title'] = df_sports['article_title'].fillna(df_sports['headline'])


In [ ]:
missing_values_table(df_sports)

Your selected dataframe has 4 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [ ]:
combined_ap_df = pd.concat([df_sports,df_politics,df_entertainment,df_business], ignore_index=True)

In [ ]:
combined_ap_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   headline         78 non-null     object
 1   article_url      78 non-null     object
 2   article_title    78 non-null     object
 3   article_content  78 non-null     object
dtypes: object(4)
memory usage: 2.6+ KB


In [ ]:
combined_ap_df.shape

(78, 4)

In [ ]:
missing_values_table(combined_ap_df)

Your selected dataframe has 4 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


## BBC

In [ ]:
# BUSINESS
df_business_bbc= pd.read_csv('/content/drive/MyDrive/web_scraped_data/BBC/bbc_business.csv')

In [ ]:
df_business_bbc.head()

,headline,article_link,article_title,article_content
0,US and China to start talks over trade war thi...,https://www.bbc.com/news/articles/c8rgrejkvmjo,US and China to start talks over trade war thi...,US and Chinese officials are set to start talk...
1,UK and India agree trade deal after three year...,https://www.bbc.com/news/articles/c5y6y90e5vzo,UK and India agree trade deal after three year...,NaN
2,NaN,https://cloud.email.bbc.com/WorldofBusiness_Ne...,NaN,NaN
3,NaN,https://www.bbc.com/news/articles/c5y6y90e5vzo,UK and India agree trade deal after three year...,The UK and India have agreed a trade deal that...
4,Analysis: UK-India deal secured as Trump tarif...,https://www.bbc.com/news/articles/cn919g2v3w2o,Faisal Islam: Trump tariffs may have helped dr...,It has long been the great prize for Britain's...


In [ ]:
missing_values_table(df_business_bbc)

Your selected dataframe has 4 columns.
There are 3 columns that have missing values.


,Missing Values,% of Total Values
article_content,4,20.0
article_title,3,15.0
headline,2,10.0


In [ ]:
df_business_bbc['article_content'] = df_business_bbc['article_content'].fillna(df_business_bbc['headline'])
df_business_bbc['article_title'] = df_business_bbc['article_title'].fillna(df_business_bbc['headline'])
df_business_bbc['headline'] = df_business_bbc['headline'].fillna(df_business_bbc['article_title'])


In [ ]:
missing_values_table(df_business_bbc)

Your selected dataframe has 4 columns.
There are 3 columns that have missing values.


,Missing Values,% of Total Values
headline,1,5.0
article_title,1,5.0
article_content,1,5.0


In [ ]:
df_business_bbc = df_business_bbc.dropna()

In [ ]:
missing_values_table(df_business_bbc)

Your selected dataframe has 4 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [ ]:
#Culture
df_culture_bbc= pd.read_csv('/content/drive/MyDrive/web_scraped_data/BBC/bbc_culture.csv')


In [ ]:
df_culture_bbc.head()

,headline,article_link,article_title,article_content
0,Call the Midwife film and prequel series annou...,https://www.bbc.com/news/articles/c93g3ldd20vo,Call the Midwife film and prequel series annou...,The BBC has announced plans to make a Call the...
1,King Charles and Queen Camilla unveil Coronati...,https://www.bbc.com/news/articles/cd020z0dl2eo,King and Queen unveil Coronation portraits,NaN
2,27 of the best looks from Met Gala 2025,https://www.bbc.com/news/articles/c20xvd10xv5o,27 of the best looks from Met Gala 2025,Monday night marked one of the world's biggest...
3,Home of Ukrainian Eurovision contestant destroyed,https://www.bbc.com/news/articles/c93g03g71qno,Home of Ukrainian Eurovision contestant destroyed,The home of Ukrainian Eurovision contestant Kh...
4,Oscar winner Vikander returns to stage after 1...,https://www.bbc.com/news/articles/cly2jjg10xlo,Oscar winner Vikander returns to stage after 1...,Actress Alicia Vikander is set to make her UK ...


In [ ]:
df_culture_bbc['article_content'] = df_culture_bbc['article_content'].fillna(df_culture_bbc['headline'])
df_culture_bbc['article_title'] = df_culture_bbc['article_title'].fillna(df_culture_bbc['headline'])
df_culture_bbc['headline'] = df_culture_bbc['headline'].fillna(df_culture_bbc['article_title'])


In [ ]:
missing_values_table(df_culture_bbc)

Your selected dataframe has 4 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [ ]:
#politics
df_politics_bbc= pd.read_csv('/content/drive/MyDrive/web_scraped_data/BBC/bbc_culture.csv')


In [ ]:
df_politics_bbc.head()

,headline,article_link,article_title,article_content
0,Call the Midwife film and prequel series annou...,https://www.bbc.com/news/articles/c93g3ldd20vo,Call the Midwife film and prequel series annou...,The BBC has announced plans to make a Call the...
1,King Charles and Queen Camilla unveil Coronati...,https://www.bbc.com/news/articles/cd020z0dl2eo,King and Queen unveil Coronation portraits,NaN
2,27 of the best looks from Met Gala 2025,https://www.bbc.com/news/articles/c20xvd10xv5o,27 of the best looks from Met Gala 2025,Monday night marked one of the world's biggest...
3,Home of Ukrainian Eurovision contestant destroyed,https://www.bbc.com/news/articles/c93g03g71qno,Home of Ukrainian Eurovision contestant destroyed,The home of Ukrainian Eurovision contestant Kh...
4,Oscar winner Vikander returns to stage after 1...,https://www.bbc.com/news/articles/cly2jjg10xlo,Oscar winner Vikander returns to stage after 1...,Actress Alicia Vikander is set to make her UK ...


In [ ]:
missing_values_table(df_politics_bbc)

Your selected dataframe has 4 columns.
There are 1 columns that have missing values.


,Missing Values,% of Total Values
article_content,1,6.2


In [ ]:
df_politics_bbc = df_politics_bbc.dropna()

In [ ]:
missing_values_table(df_politics_bbc)

Your selected dataframe has 4 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [ ]:
#sports
df_sports_bbc= pd.read_csv('/content/drive/MyDrive/web_scraped_data/BBC/bbc_sports.csv')

In [ ]:
df_sports_bbc.head()

,headline,article_link,article_title,article_content
0,LIVE . Champions League reaction as Inter beat...,https://www.bbc.com/sport/football/live/cp8k81...,Champions League reaction as Inter beat Barcel...,Champions League reaction as Inter beat Barcel...
1,'Thank you Inter & Barca' - the game nobody wa...,https://www.bbc.com/sport/football/articles/c2...,'Thank you Inter & Barca' - the game nobody wa...,Inter Milan will face Paris St-Germain or Arse...
2,"'A liability', 'the aura' & 'a phenomenon' - t...",https://www.bbc.com/sport/rugby-union/articles...,"'A liability', 'the aura' & 'a phenomenon' - t...",Head coach Andy Farrell (third from left) and ...
3,Inter Miami relinquish option to speak to De B...,https://www.bbc.com/sport/football/articles/c2...,Inter Miami relinquish option to speak to De B...,Kevin de Bruyne has scored 108 goals in 418 ap...
4,Where does Inter v Barca rank among best-ever ...,https://www.bbc.com/sport/football/articles/c9...,Where does Inter v Barca rank among best-ever ...,In what is already being talked about as one o...


In [ ]:
missing_values_table(df_sports_bbc)

Your selected dataframe has 4 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [ ]:
combined_bbc_df = pd.concat([df_sports_bbc,df_politics_bbc,df_culture_bbc,df_culture_bbc], ignore_index=True)

In [ ]:
combined_bbc_df.shape

(67, 4)

In [ ]:
missing_values_table(combined_bbc_df)

Your selected dataframe has 4 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


#  CBS

In [ ]:
# Business

df_business_cbs= pd.read_csv('/content/drive/MyDrive/web_scraped_data/CBS/cbsnews_business.csv')


In [ ]:
df_business_cbs.head()

,headline,article_url,article_title,article_content,published_date,writer
0,These Chinese-made products could soon be hard...,https://www.cbsnews.com/news/chinese-china-pro...,These Chinese-made products could soon be hard...,Shipments of Chinese goods to the U.S. are plu...,"Updated on: May 6, 2025 / 1:57 PM EDT / CBS News",Megan Cerullo
1,Flying without a Real ID? Here's what to know ...,https://www.cbsnews.com/news/real-id-flying-ai...,Flying without a Real ID? Here's what to know ...,It's a nightmare for any traveler: getting to ...,"May 6, 2025 / 4:45 PM EDT / CBS News",Alex Sundby
2,Trump wants the Fed to cut rates on Wednesday....,https://www.cbsnews.com/news/trump-fed-rate-cu...,Trump wants the Fed to cut rates on Wednesday....,President Trump has been pushing the Federal R...,"Updated on: May 6, 2025 / 1:58 PM EDT / CBS News",Aimee Picchi
3,Grand Theft Auto 6 drops a surprise second tra...,https://www.cbsnews.com/news/gta-6-trailer-2-r...,Grand Theft Auto 6 drops a surprise second tra...,Grand Theft Auto 6 has dropped a surprise seco...,"Updated on: May 6, 2025 / 1:45 PM EDT / CBS News",Aimee Picchi
4,Flowers for Mother's Day may be more expensive...,https://www.cbsnews.com/news/flower-prices-mot...,Flowers for Mother's Day may be more expensive...,"With Mother's Day approaching, you may be look...","May 6, 2025 / 5:10 PM EDT / CBS News",NaN


In [ ]:
df_business_cbs.drop(columns=['published_date','writer'], inplace=True)

In [ ]:
df_business_cbs.head()

,headline,article_url,article_title,article_content
0,These Chinese-made products could soon be hard...,https://www.cbsnews.com/news/chinese-china-pro...,These Chinese-made products could soon be hard...,Shipments of Chinese goods to the U.S. are plu...
1,Flying without a Real ID? Here's what to know ...,https://www.cbsnews.com/news/real-id-flying-ai...,Flying without a Real ID? Here's what to know ...,It's a nightmare for any traveler: getting to ...
2,Trump wants the Fed to cut rates on Wednesday....,https://www.cbsnews.com/news/trump-fed-rate-cu...,Trump wants the Fed to cut rates on Wednesday....,President Trump has been pushing the Federal R...
3,Grand Theft Auto 6 drops a surprise second tra...,https://www.cbsnews.com/news/gta-6-trailer-2-r...,Grand Theft Auto 6 drops a surprise second tra...,Grand Theft Auto 6 has dropped a surprise seco...
4,Flowers for Mother's Day may be more expensive...,https://www.cbsnews.com/news/flower-prices-mot...,Flowers for Mother's Day may be more expensive...,"With Mother's Day approaching, you may be look..."


In [ ]:
missing_values_table(df_business_cbs)

Your selected dataframe has 4 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [ ]:
# Entertainment

df_entertainment_cbs= pd.read_csv('/content/drive/MyDrive/web_scraped_data/CBS/cbsnews_entertainment.csv')


In [ ]:
df_entertainment_cbs.head()

,headline,article_url,article_title,article_content,published_date,writer
0,"Tinx tackles cancel culture, romance in debut ...",https://www.cbsnews.com/news/tinx-hotter-in-th...,Tinx tackles cancel culture and romance in deb...,Social media star and bestselling author Chris...,"May 6, 2025 / 1:05 PM EDT / CBS News",Analisa Novak
1,"Kamala Harris makes Met Gala debut, says art a...",https://www.cbsnews.com/news/kamala-harris-met...,"Kamala Harris makes Met Gala debut, says art a...",Former Vice President Kamala Harris made an ap...,"May 5, 2025 / 11:19 PM EDT / CBS News",Kiki Intarasuwan
2,A$AP Rocky confirms baby No. 3 with Rihanna at...,https://www.cbsnews.com/news/rihanna-pregnant-...,A$AP Rocky confirms baby No. 3 with Rihanna at...,Rihanna and A$AP Rocky once again stole the sh...,"Updated on: May 5, 2025 / 10:23 PM EDT / CBS/AP",NaN
3,Diana Ross stuns Met Gala blue carpet with 18-...,https://www.cbsnews.com/news/diana-ross-met-ga...,Diana Ross stuns Met Gala blue carpet with 18-...,Diana Ross graced the blue carpet of the Metro...,"Updated on: May 5, 2025 / 8:07 PM EDT / CBS News",Kiki Intarasuwan
4,Here's how much it costs to attend the 2025 Me...,https://www.cbsnews.com/news/2025-met-gala-tic...,How much does it cost to attend the 2025 Met G...,Inflation has pushed up the price of many thin...,"Updated on: May 5, 2025 / 11:57 AM EDT / CBS News",Megan Cerullo


In [ ]:
df_entertainment_cbs.drop(columns=['published_date','writer'], inplace=True)

In [ ]:
df_entertainment_cbs.head()

,headline,article_url,article_title,article_content
0,"Tinx tackles cancel culture, romance in debut ...",https://www.cbsnews.com/news/tinx-hotter-in-th...,Tinx tackles cancel culture and romance in deb...,Social media star and bestselling author Chris...
1,"Kamala Harris makes Met Gala debut, says art a...",https://www.cbsnews.com/news/kamala-harris-met...,"Kamala Harris makes Met Gala debut, says art a...",Former Vice President Kamala Harris made an ap...
2,A$AP Rocky confirms baby No. 3 with Rihanna at...,https://www.cbsnews.com/news/rihanna-pregnant-...,A$AP Rocky confirms baby No. 3 with Rihanna at...,Rihanna and A$AP Rocky once again stole the sh...
3,Diana Ross stuns Met Gala blue carpet with 18-...,https://www.cbsnews.com/news/diana-ross-met-ga...,Diana Ross stuns Met Gala blue carpet with 18-...,Diana Ross graced the blue carpet of the Metro...
4,Here's how much it costs to attend the 2025 Me...,https://www.cbsnews.com/news/2025-met-gala-tic...,How much does it cost to attend the 2025 Met G...,Inflation has pushed up the price of many thin...


In [ ]:
missing_values_table(df_entertainment_cbs)

Your selected dataframe has 4 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [ ]:
# political
df_political_cbs= pd.read_csv('/content/drive/MyDrive/web_scraped_data/CBS/cbsnews_politics.csv')


In [ ]:
missing_values_table(df_political_cbs)


Your selected dataframe has 6 columns.
There are 1 columns that have missing values.


,Missing Values,% of Total Values
writer,4,16.0


In [ ]:
df_political_cbs.drop(columns=['published_date','writer'], inplace=True)

In [ ]:
missing_values_table(df_political_cbs)

Your selected dataframe has 4 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [ ]:
#Sports

df_sports_cbs= pd.read_csv('/content/drive/MyDrive/web_scraped_data/CBS/cbs_sports.csv')

In [ ]:
df_sports_cbs.head()

,headline,article_url,article_title,article_content,published_date,writer
0,Raleigh's pinch-hit single in 9th sends Marine...,https://www.cbsnews.com/sanfrancisco/news/rale...,Raleigh's pinch-hit single in 9th sends Marine...,WEST SACRAMENTO — Pinch-hitter Cal Raleigh put...,"May 6, 2025 / 11:23 PM PDT / AP",NaN
1,Valkyries fall short in second half comeback l...,https://www.cbsnews.com/sanfrancisco/news/valk...,Valkyries fall short after unsuccessful second...,"In their first-ever game, the Golden State Val...","Updated on: May 6, 2025 / 10:13 PM PDT / CBS S...",Jose Fabian
2,Warriors take Game 1 from Wolves 99-88 Buddy H...,https://www.cbsnews.com/sanfrancisco/news/warr...,Wolves struggle to keep up with the Warriors i...,Buddy Hield and Draymond Green delivered from ...,"May 6, 2025 / 9:29 PM PDT / AP",NaN
3,How to watch the Golden State Valkyries' first...,https://www.cbsnews.com/sanfrancisco/news/how-...,How to watch the Golden State Valkyries' first...,"On Tuesday night, the Golden State Valkyries w...","Updated on: May 6, 2025 / 2:18 PM PDT / CBS Sa...",NaN
4,Sharks to pick 2nd in 2025 NHL Draft following...,https://www.cbsnews.com/sanfrancisco/news/2025...,Sharks to pick 2nd in 2025 NHL Draft following...,New York Islanders pro scouting director Ken M...,"Updated on: May 6, 2025 / 12:16 AM PDT / AP",NaN


In [ ]:
df_sports_cbs.drop(columns=['published_date','writer'], inplace=True)

In [ ]:
missing_values_table(df_sports_cbs)

Your selected dataframe has 4 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [ ]:
df_sports_cbs.head()

,headline,article_url,article_title,article_content
0,Raleigh's pinch-hit single in 9th sends Marine...,https://www.cbsnews.com/sanfrancisco/news/rale...,Raleigh's pinch-hit single in 9th sends Marine...,WEST SACRAMENTO — Pinch-hitter Cal Raleigh put...
1,Valkyries fall short in second half comeback l...,https://www.cbsnews.com/sanfrancisco/news/valk...,Valkyries fall short after unsuccessful second...,"In their first-ever game, the Golden State Val..."
2,Warriors take Game 1 from Wolves 99-88 Buddy H...,https://www.cbsnews.com/sanfrancisco/news/warr...,Wolves struggle to keep up with the Warriors i...,Buddy Hield and Draymond Green delivered from ...
3,How to watch the Golden State Valkyries' first...,https://www.cbsnews.com/sanfrancisco/news/how-...,How to watch the Golden State Valkyries' first...,"On Tuesday night, the Golden State Valkyries w..."
4,Sharks to pick 2nd in 2025 NHL Draft following...,https://www.cbsnews.com/sanfrancisco/news/2025...,Sharks to pick 2nd in 2025 NHL Draft following...,New York Islanders pro scouting director Ken M...


In [ ]:
combined_cbs_df = pd.concat([df_political_cbs,df_entertainment_cbs,df_business_cbs,df_sports_cbs], ignore_index=True)

In [ ]:
combined_cbs_df.head()

,headline,article_url,article_title,article_content
0,"Seeking spending cuts, GOP targets a tax hospi...",https://www.cbsnews.com/news/gop-spending-cuts...,"Seeking spending cuts, GOP lawmakers target a ...","On the eastern plains of Colorado, in a county..."
1,Trump slams Newsom after California governor p...,https://www.cbsnews.com/news/president-trump-g...,Trump slams Newsom after California governor p...,President Trump slammed Gavin Newsom on Tuesda...
2,Trump administration may soon deport migrants ...,https://www.cbsnews.com/news/trump-administrat...,Trump administration may soon deport migrants ...,The Trump administration may soon start deport...
3,"For second time in 8 days, fighter jet assigne...",https://www.cbsnews.com/news/u-s-fighter-jet-a...,"For second time in 8 days, U.S. fighter jet as...","For the second time in just over a week, an F/..."
4,Trump says fewer than 2 dozen hostages believe...,https://www.cbsnews.com/news/trump-says-less-t...,Trump says fewer than 2 dozen hostages believe...,President Trump said Tuesday that three hostag...


In [ ]:
combined_cbs_df.shape

(100, 4)

# NBC

In [ ]:
df_sports_nbc = pd.read_csv('/content/drive/MyDrive/web_scraped_data/NBC/nbc_sports_stories.csv')

In [ ]:
df_sports_nbc.head()

,headline,description,timestamp,article_link,article_content,writer,date_published
0,"Until NFL investigation ends, Justin Tucker co...",The Ravens have released kicker Justin Tucker....,NaN,https://www.nbcsports.com/nfl/profootballtalk/...,The Ravens have released kicker Justin Tucker....,NaN,"Published May 6, 2025 11:33 AM"
1,Sports Icons Shine at the 2025 Met Gala: Angel...,The biggest names in sports took the red carpe...,NaN,https://www.nbcsports.com/wnba/news/sports-ico...,Some would equate the Met Gala to “the Super B...,NaN,"Published May 6, 2025 11:26 AM"
2,"Ted Cruz: NFL has been ""tiptoeing up to the ru...",Hearing on broadcast vs. streaming could spark...,NaN,https://www.nbcsports.com/nfl/profootballtalk/...,The Senate Commerce Committee has proceeded wi...,NaN,"Published May 6, 2025 10:35 AM"
3,Billy Horschel's Ryder Cup dreams likely dashe...,Horschel revealed Tuesday that he will be side...,NaN,https://www.nbcsports.com/golf/news/billy-hors...,Billy Horschel’s Ryder Cup dreams are likely d...,NaN,"Published May 6, 2025 10:17 AM"
4,Nylander has 2 goals and an assist as Maple Le...,Matthew Knies and Chris Tanev each had a goal ...,NaN,https://www.nbcsports.com/nhl/news/nylander-ha...,John E. Sokolowski-Imagn Images TORONTO (AP) W...,NaN,"Published May 5, 2025 11:51 PM"


In [ ]:
df_sports_nbc.drop(columns=['date_published','writer'], inplace=True)

In [ ]:
df_sports_nbc.head()

,headline,description,timestamp,article_link,article_content
0,"Until NFL investigation ends, Justin Tucker co...",The Ravens have released kicker Justin Tucker....,NaN,https://www.nbcsports.com/nfl/profootballtalk/...,The Ravens have released kicker Justin Tucker....
1,Sports Icons Shine at the 2025 Met Gala: Angel...,The biggest names in sports took the red carpe...,NaN,https://www.nbcsports.com/wnba/news/sports-ico...,Some would equate the Met Gala to “the Super B...
2,"Ted Cruz: NFL has been ""tiptoeing up to the ru...",Hearing on broadcast vs. streaming could spark...,NaN,https://www.nbcsports.com/nfl/profootballtalk/...,The Senate Commerce Committee has proceeded wi...
3,Billy Horschel's Ryder Cup dreams likely dashe...,Horschel revealed Tuesday that he will be side...,NaN,https://www.nbcsports.com/golf/news/billy-hors...,Billy Horschel’s Ryder Cup dreams are likely d...
4,Nylander has 2 goals and an assist as Maple Le...,Matthew Knies and Chris Tanev each had a goal ...,NaN,https://www.nbcsports.com/nhl/news/nylander-ha...,John E. Sokolowski-Imagn Images TORONTO (AP) W...


In [ ]:
df_sports_nbc.rename(columns={'article_link': 'article_url'}, inplace=True)
df_sports_nbc.rename(columns={'description': 'article_title'}, inplace=True)


In [ ]:
missing_values_table(df_sports_nbc)


Your selected dataframe has 4 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [ ]:
df_sports_nbc.drop(columns=['timestamp'], inplace=True)


In [ ]:
missing_values_table(df_sports_nbc)

Your selected dataframe has 4 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [ ]:
df_sports_nbc.head()

,headline,article_title,article_url,article_content
0,"Until NFL investigation ends, Justin Tucker co...",The Ravens have released kicker Justin Tucker....,https://www.nbcsports.com/nfl/profootballtalk/...,The Ravens have released kicker Justin Tucker....
1,Sports Icons Shine at the 2025 Met Gala: Angel...,The biggest names in sports took the red carpe...,https://www.nbcsports.com/wnba/news/sports-ico...,Some would equate the Met Gala to “the Super B...
2,"Ted Cruz: NFL has been ""tiptoeing up to the ru...",Hearing on broadcast vs. streaming could spark...,https://www.nbcsports.com/nfl/profootballtalk/...,The Senate Commerce Committee has proceeded wi...
3,Billy Horschel's Ryder Cup dreams likely dashe...,Horschel revealed Tuesday that he will be side...,https://www.nbcsports.com/golf/news/billy-hors...,Billy Horschel’s Ryder Cup dreams are likely d...
4,Nylander has 2 goals and an assist as Maple Le...,Matthew Knies and Chris Tanev each had a goal ...,https://www.nbcsports.com/nhl/news/nylander-ha...,John E. Sokolowski-Imagn Images TORONTO (AP) W...


In [ ]:
# Business
df_business_nbc= pd.read_csv('/content/drive/MyDrive/web_scraped_data/NBC/nbc_business.csv')


In [ ]:
df_business_nbc.head()

,headline,description,timestamp,article_link,article_content,writer,date_published
0,How GM is trying to revive Cadillac as the qui...,The luxury vehicle market is crucial for autom...,NaN,https://www.nbcnews.com/business/autos/gms-dec...,"WARREN, Mich. — Walking into General Motors’ g...","Michael Wayland, CNBC","May 5, 2025, 7:28 PM GMT+2 / Updated May 6, 20..."
1,Air traffic control at Newark Airport briefly ...,"After the incident last week, the controllers ...",NaN,https://www.nbcnews.com/news/us-news/air-traff...,Air traffic controllers temporarily lost commu...,Mirna Alsharif,"May 5, 2025, 10:34 PM GMT+2 / Updated May 6, 2..."
2,Trump's Hollywood tariffs announcement is met ...,A host of questions remain about how the presi...,NaN,https://www.nbcnews.com/business/business-news...,President Donald Trump’s Sunday night proposal...,Rob Wile,"May 5, 2025, 7:13 PM GMT+2 / Updated May 5, 20..."
3,Social Security reduces benefit clawback from ...,Beneficiaries may owe the Social Security Admi...,NaN,https://www.nbcnews.com/business/consumer/soci...,Just weeks after announcing a 100% withholding...,"Lorie Konish, CNBC","May 5, 2025, 11:08 PM GMT+2 / Source: CNBC"
4,Rite Aid files for second bankruptcy in two years,Rite Aid used its previous bankruptcy in 2023 ...,NaN,https://www.nbcnews.com/business/business-news...,U.S. pharmacy chain Rite Aid on Monday filed f...,Reuters,"May 5, 2025, 10:56 PM GMT+2 / Source: Reuters"


In [ ]:
df_business_nbc.rename(columns={'article_link': 'article_url'}, inplace=True)
df_business_nbc.rename(columns={'description': 'article_title'}, inplace=True)
df_business_nbc.drop(columns=['timestamp'], inplace=True)

In [ ]:
df_business_nbc.drop(columns=['date_published','writer'], inplace=True)

In [ ]:
missing_values_table(df_sports_nbc)

Your selected dataframe has 4 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [ ]:
df_business_nbc.head()

,headline,article_title,article_url,article_content
0,How GM is trying to revive Cadillac as the qui...,The luxury vehicle market is crucial for autom...,https://www.nbcnews.com/business/autos/gms-dec...,"WARREN, Mich. — Walking into General Motors’ g..."
1,Air traffic control at Newark Airport briefly ...,"After the incident last week, the controllers ...",https://www.nbcnews.com/news/us-news/air-traff...,Air traffic controllers temporarily lost commu...
2,Trump's Hollywood tariffs announcement is met ...,A host of questions remain about how the presi...,https://www.nbcnews.com/business/business-news...,President Donald Trump’s Sunday night proposal...
3,Social Security reduces benefit clawback from ...,Beneficiaries may owe the Social Security Admi...,https://www.nbcnews.com/business/consumer/soci...,Just weeks after announcing a 100% withholding...
4,Rite Aid files for second bankruptcy in two years,Rite Aid used its previous bankruptcy in 2023 ...,https://www.nbcnews.com/business/business-news...,U.S. pharmacy chain Rite Aid on Monday filed f...


In [ ]:
#Politics
df_politics_nbc= pd.read_csv('/content/drive/MyDrive/web_scraped_data/NBC/nbc_politics_stories.csv')

In [ ]:
df_politics_nbc.drop(columns=['date_published','writer'], inplace=True)
df_politics_nbc.drop(columns=['timestamp'], inplace=True)

In [ ]:
missing_values_table(df_sports_nbc)

Your selected dataframe has 4 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [ ]:
df_politics_nbc.head()

,headline,description,article_link,article_content
0,Key GOP senator won't support Trump loyalist E...,North Carolina Republican Sen. Thom Tillis' de...,https://www.nbcnews.com/politics/justice-depar...,"WASHINGTON — Sen. Thom Tillis, R-N.C., said he..."
1,"Angus King III, son of the Maine senator, laun...",King is seeking to succeed term-limited Gov. J...,https://www.nbcnews.com/politics/2026-election...,"Angus King III, an energy executive and a son ..."
2,Trump says D.C. will host the 2027 NFL draft; ...,Trump also discussed his plans to impose tarif...,https://www.nbcnews.com/politics/trump-adminis...,Raquel Coronell Uribe In a Truth Social post t...
3,Federal judge says results of North Carolina c...,More than 5.5 million ballots were cast in wha...,https://www.nbcnews.com/politics/elections/fed...,"RALEIGH, N.C. — Disputed ballots in the still ..."
4,Alexandria Ocasio-Cortez won't seek the top De...,"After speaking to colleagues, the progressive ...",https://www.nbcnews.com/politics/congress/aoc-...,WASHINGTON — Rep. Alexandria Ocasio-Cortez sai...


In [ ]:
df_politics_nbc.rename(columns={'article_link': 'article_url'}, inplace=True)
df_politics_nbc.rename(columns={'description': 'article_title'}, inplace=True)


In [ ]:
df_politics_nbc.head()

,headline,article_title,article_url,article_content
0,Key GOP senator won't support Trump loyalist E...,North Carolina Republican Sen. Thom Tillis' de...,https://www.nbcnews.com/politics/justice-depar...,"WASHINGTON — Sen. Thom Tillis, R-N.C., said he..."
1,"Angus King III, son of the Maine senator, laun...",King is seeking to succeed term-limited Gov. J...,https://www.nbcnews.com/politics/2026-election...,"Angus King III, an energy executive and a son ..."
2,Trump says D.C. will host the 2027 NFL draft; ...,Trump also discussed his plans to impose tarif...,https://www.nbcnews.com/politics/trump-adminis...,Raquel Coronell Uribe In a Truth Social post t...
3,Federal judge says results of North Carolina c...,More than 5.5 million ballots were cast in wha...,https://www.nbcnews.com/politics/elections/fed...,"RALEIGH, N.C. — Disputed ballots in the still ..."
4,Alexandria Ocasio-Cortez won't seek the top De...,"After speaking to colleagues, the progressive ...",https://www.nbcnews.com/politics/congress/aoc-...,WASHINGTON — Rep. Alexandria Ocasio-Cortez sai...


In [ ]:
#Culture
df_culture_nbc= pd.read_csv('/content/drive/MyDrive/web_scraped_data/NBC/nbc_arts_culture_and_celebrities.csv')

In [ ]:
df_culture_nbc.rename(columns={'article_link': 'article_url'}, inplace=True)
df_culture_nbc.rename(columns={'description': 'article_title'}, inplace=True)
df_culture_nbc.drop(columns=['timestamp'], inplace=True)


In [ ]:
df_culture_nbc.head()

,headline,article_title,article_url,article_content,writer,date_published
0,Lady Gaga says she was unaware of thwarted bom...,Unidentified people were “virtually recruiting...,https://www.nbcnews.com/pop-culture/pop-cultur...,A planned bombing at a Lady Gaga concert in Br...,Doha Madani,"May 4, 2025, 5:44 PM GMT+2 / Updated May 4, 20..."
1,"Ruth Buzzi, comedy sketch player on groundbrea...",Buzzi rose to fame as the frumpy and bitter Gl...,https://www.nbcnews.com/pop-culture/pop-cultur...,"LOS ANGELES — Ruth Buzzi, who rose to fame as ...",The Associated Press,"May 2, 2025, 5:43 PM GMT+2 / Updated May 2, 20..."
2,Blake Lively said past year filled with 'lowes...,The “Another Simple Favor” star talked about h...,https://www.nbcnews.com/pop-culture/pop-cultur...,Blake Lively is speaking out about what she’s ...,"Liz Calvario, TODAY","May 2, 2025, 5:02 PM GMT+2 / Source: TODAY"
3,"Singer Jill Sobule, known for 'I Kissed a Girl...",Sobule was 66. The 1995 song was the first ope...,https://www.nbcnews.com/pop-culture/music/jill...,"Singer Jill Sobule, known for the 1995 song “I...",Phil Helsel,"May 2, 2025, 3:29 AM GMT+2 / Updated May 2, 20..."
4,Video shows Justin Timberlake becoming Travis ...,Timberlake posted a clip of the impromptu perf...,https://www.nbcnews.com/pop-culture/pop-cultur...,Boy band veteran Justin Timberlake brought his...,Viola Flowers,"May 1, 2025, 6:01 PM GMT+2"


In [ ]:
df_culture_nbc.drop(columns=['date_published','writer'], inplace=True)


In [ ]:
df_culture_nbc.head()

,headline,article_title,article_url,article_content
0,Lady Gaga says she was unaware of thwarted bom...,Unidentified people were “virtually recruiting...,https://www.nbcnews.com/pop-culture/pop-cultur...,A planned bombing at a Lady Gaga concert in Br...
1,"Ruth Buzzi, comedy sketch player on groundbrea...",Buzzi rose to fame as the frumpy and bitter Gl...,https://www.nbcnews.com/pop-culture/pop-cultur...,"LOS ANGELES — Ruth Buzzi, who rose to fame as ..."
2,Blake Lively said past year filled with 'lowes...,The “Another Simple Favor” star talked about h...,https://www.nbcnews.com/pop-culture/pop-cultur...,Blake Lively is speaking out about what she’s ...
3,"Singer Jill Sobule, known for 'I Kissed a Girl...",Sobule was 66. The 1995 song was the first ope...,https://www.nbcnews.com/pop-culture/music/jill...,"Singer Jill Sobule, known for the 1995 song “I..."
4,Video shows Justin Timberlake becoming Travis ...,Timberlake posted a clip of the impromptu perf...,https://www.nbcnews.com/pop-culture/pop-cultur...,Boy band veteran Justin Timberlake brought his...


In [ ]:
missing_values_table(df_sports_nbc)

Your selected dataframe has 4 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [ ]:
combined_nbc_dfcombined_nbc_df = pd.concat([df_culture_nbc,df_politics_nbc,df_business_nbc,df_sports_nbc], ignore_index=True)

In [ ]:
missing_values_table(combined_nbc_df)

Your selected dataframe has 4 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [ ]:
combined_nbc_df.shape

(80, 4)

In [ ]:
combined_nbc_df

,headline,article_title,article_url,article_content
0,Lady Gaga says she was unaware of thwarted bom...,Unidentified people were “virtually recruiting...,https://www.nbcnews.com/pop-culture/pop-cultur...,A planned bombing at a Lady Gaga concert in Br...
1,"Ruth Buzzi, comedy sketch player on groundbrea...",Buzzi rose to fame as the frumpy and bitter Gl...,https://www.nbcnews.com/pop-culture/pop-cultur...,"LOS ANGELES — Ruth Buzzi, who rose to fame as ..."
2,Blake Lively said past year filled with 'lowes...,The “Another Simple Favor” star talked about h...,https://www.nbcnews.com/pop-culture/pop-cultur...,Blake Lively is speaking out about what she’s ...
3,"Singer Jill Sobule, known for 'I Kissed a Girl...",Sobule was 66. The 1995 song was the first ope...,https://www.nbcnews.com/pop-culture/music/jill...,"Singer Jill Sobule, known for the 1995 song “I..."
4,Video shows Justin Timberlake becoming Travis ...,Timberlake posted a clip of the impromptu perf...,https://www.nbcnews.com/pop-culture/pop-cultur...,Boy band veteran Justin Timberlake brought his...
...,...,...,...,...
75,"Matthew Stafford, Rams will have another early...","In the NFL, no deal is ever done until it's do...",https://www.nbcsports.com/nfl/profootballtalk/...,"In the NFL, no deal is ever done until it’s do..."
76,NFL declines comment on proposed White House c...,Commissioner Roger Goodell went to Washington ...,https://www.nbcsports.com/nfl/profootballtalk/...,Commissioner Roger Goodell went to Washington ...
77,"Ravens made a ""football decision"" (wink, nod) ...","P.R. fallout be damned, the team carefully avo...",https://www.nbcsports.com/nfl/profootballtalk/...,The Ravens have been in a delicate spot from t...
78,Ravens will designate Justin Tucker as a post-...,The Ravens announced Monday they are releasing...,https://www.nbcsports.com/nfl/profootballtalk/...,The Ravens announced Monday they are releasing...


In [ ]:
webscraped_dataset = pd.concat([combined_nbc_df,combined_ap_df ,combined_cbs_df,combined_bbc_df], ignore_index=True)


In [ ]:
webscraped_dataset.head()

,headline,article_title,article_url,article_content,article_link
0,Lady Gaga says she was unaware of thwarted bom...,Unidentified people were “virtually recruiting...,https://www.nbcnews.com/pop-culture/pop-cultur...,A planned bombing at a Lady Gaga concert in Br...,NaN
1,"Ruth Buzzi, comedy sketch player on groundbrea...",Buzzi rose to fame as the frumpy and bitter Gl...,https://www.nbcnews.com/pop-culture/pop-cultur...,"LOS ANGELES — Ruth Buzzi, who rose to fame as ...",NaN
2,Blake Lively said past year filled with 'lowes...,The “Another Simple Favor” star talked about h...,https://www.nbcnews.com/pop-culture/pop-cultur...,Blake Lively is speaking out about what she’s ...,NaN
3,"Singer Jill Sobule, known for 'I Kissed a Girl...",Sobule was 66. The 1995 song was the first ope...,https://www.nbcnews.com/pop-culture/music/jill...,"Singer Jill Sobule, known for the 1995 song “I...",NaN
4,Video shows Justin Timberlake becoming Travis ...,Timberlake posted a clip of the impromptu perf...,https://www.nbcnews.com/pop-culture/pop-cultur...,Boy band veteran Justin Timberlake brought his...,NaN


In [ ]:
df_business = df_business.dropna()

In [ ]:
combined_nbc_df.head()

,headline,article_title,article_url,article_content
0,Lady Gaga says she was unaware of thwarted bom...,Unidentified people were “virtually recruiting...,https://www.nbcnews.com/pop-culture/pop-cultur...,A planned bombing at a Lady Gaga concert in Br...
1,"Ruth Buzzi, comedy sketch player on groundbrea...",Buzzi rose to fame as the frumpy and bitter Gl...,https://www.nbcnews.com/pop-culture/pop-cultur...,"LOS ANGELES — Ruth Buzzi, who rose to fame as ..."
2,Blake Lively said past year filled with 'lowes...,The “Another Simple Favor” star talked about h...,https://www.nbcnews.com/pop-culture/pop-cultur...,Blake Lively is speaking out about what she’s ...
3,"Singer Jill Sobule, known for 'I Kissed a Girl...",Sobule was 66. The 1995 song was the first ope...,https://www.nbcnews.com/pop-culture/music/jill...,"Singer Jill Sobule, known for the 1995 song “I..."
4,Video shows Justin Timberlake becoming Travis ...,Timberlake posted a clip of the impromptu perf...,https://www.nbcnews.com/pop-culture/pop-cultur...,Boy band veteran Justin Timberlake brought his...


In [ ]:
combined_ap_df.head()

,headline,article_url,article_title,article_content
0,Curry leaves Warriors’ win vs. Wolves with ham...,https://apnews.com/article/nba-playoffs-warrio...,Curry leaves Warriors’ win vs. Wolves with ham...,Curry leaves Warriors’ win vs. Wolves with ham...
1,Golden State star Stephen Curry was sidelined ...,https://apnews.com/article/nba-playoffs-warrio...,Curry leaves Warriors’ win vs. Wolves with ham...,MINNEAPOLIS (AP) — Golden State star Stephen C...
2,Haliburton’s 3 with 1.1 seconds left gives Pac...,https://apnews.com/article/nba-playoffs-pacers...,Haliburton’s 3 with 1.1 seconds left gives Pac...,CLEVELAND (AP) — Tyrese Haliburton said he kno...
3,Jaccob Slavin scores in OT as the Hurricanes b...,https://apnews.com/article/hurricanes-capitals...,Jaccob Slavin scores in OT as the Hurricanes b...,WASHINGTON (AP) — Very few people in the arena...
4,Oilers rally for NHL-record fifth straight tim...,https://apnews.com/article/oilers-golden-knigh...,Oilers rally for NHL-record fifth straight tim...,LAS VEGAS (AP) — One thing is becoming clear d...


In [ ]:
combined_cbs_df.head()

,headline,article_url,article_title,article_content
0,"Seeking spending cuts, GOP targets a tax hospi...",https://www.cbsnews.com/news/gop-spending-cuts...,"Seeking spending cuts, GOP lawmakers target a ...","On the eastern plains of Colorado, in a county..."
1,Trump slams Newsom after California governor p...,https://www.cbsnews.com/news/president-trump-g...,Trump slams Newsom after California governor p...,President Trump slammed Gavin Newsom on Tuesda...
2,Trump administration may soon deport migrants ...,https://www.cbsnews.com/news/trump-administrat...,Trump administration may soon deport migrants ...,The Trump administration may soon start deport...
3,"For second time in 8 days, fighter jet assigne...",https://www.cbsnews.com/news/u-s-fighter-jet-a...,"For second time in 8 days, U.S. fighter jet as...","For the second time in just over a week, an F/..."
4,Trump says fewer than 2 dozen hostages believe...,https://www.cbsnews.com/news/trump-says-less-t...,Trump says fewer than 2 dozen hostages believe...,President Trump said Tuesday that three hostag...


In [ ]:
combined_bbc_df.head()

,headline,article_url,article_title,article_content
0,LIVE . Champions League reaction as Inter beat...,https://www.bbc.com/sport/football/live/cp8k81...,Champions League reaction as Inter beat Barcel...,Champions League reaction as Inter beat Barcel...
1,'Thank you Inter & Barca' - the game nobody wa...,https://www.bbc.com/sport/football/articles/c2...,'Thank you Inter & Barca' - the game nobody wa...,Inter Milan will face Paris St-Germain or Arse...
2,"'A liability', 'the aura' & 'a phenomenon' - t...",https://www.bbc.com/sport/rugby-union/articles...,"'A liability', 'the aura' & 'a phenomenon' - t...",Head coach Andy Farrell (third from left) and ...
3,Inter Miami relinquish option to speak to De B...,https://www.bbc.com/sport/football/articles/c2...,Inter Miami relinquish option to speak to De B...,Kevin de Bruyne has scored 108 goals in 418 ap...
4,Where does Inter v Barca rank among best-ever ...,https://www.bbc.com/sport/football/articles/c9...,Where does Inter v Barca rank among best-ever ...,In what is already being talked about as one o...


In [ ]:
combined_bbc_df.rename(columns={'article_link': 'article_url'}, inplace=True)

In [ ]:
webscraped_dataset = pd.concat([combined_nbc_df,combined_ap_df ,combined_cbs_df,combined_bbc_df], ignore_index=True)


In [ ]:
webscraped_dataset.head()

,headline,article_title,article_url,article_content
0,Lady Gaga says she was unaware of thwarted bom...,Unidentified people were “virtually recruiting...,https://www.nbcnews.com/pop-culture/pop-cultur...,A planned bombing at a Lady Gaga concert in Br...
1,"Ruth Buzzi, comedy sketch player on groundbrea...",Buzzi rose to fame as the frumpy and bitter Gl...,https://www.nbcnews.com/pop-culture/pop-cultur...,"LOS ANGELES — Ruth Buzzi, who rose to fame as ..."
2,Blake Lively said past year filled with 'lowes...,The “Another Simple Favor” star talked about h...,https://www.nbcnews.com/pop-culture/pop-cultur...,Blake Lively is speaking out about what she’s ...
3,"Singer Jill Sobule, known for 'I Kissed a Girl...",Sobule was 66. The 1995 song was the first ope...,https://www.nbcnews.com/pop-culture/music/jill...,"Singer Jill Sobule, known for the 1995 song “I..."
4,Video shows Justin Timberlake becoming Travis ...,Timberlake posted a clip of the impromptu perf...,https://www.nbcnews.com/pop-culture/pop-cultur...,Boy band veteran Justin Timberlake brought his...


In [ ]:
missing_values_table(webscraped_dataset)

Your selected dataframe has 4 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [ ]:
webscraped_dataset.to_csv("webscraped_dataset.csv", index=False)


In [291]:
webscraped_dataset


,headline,article_title,article_url,article_content
0,Lady Gaga says she was unaware of thwarted bom...,Unidentified people were “virtually recruiting...,https://www.nbcnews.com/pop-culture/pop-cultur...,A planned bombing at a Lady Gaga concert in Br...
1,"Ruth Buzzi, comedy sketch player on groundbrea...",Buzzi rose to fame as the frumpy and bitter Gl...,https://www.nbcnews.com/pop-culture/pop-cultur...,"LOS ANGELES — Ruth Buzzi, who rose to fame as ..."
2,Blake Lively said past year filled with 'lowes...,The “Another Simple Favor” star talked about h...,https://www.nbcnews.com/pop-culture/pop-cultur...,Blake Lively is speaking out about what she’s ...
3,"Singer Jill Sobule, known for 'I Kissed a Girl...",Sobule was 66. The 1995 song was the first ope...,https://www.nbcnews.com/pop-culture/music/jill...,"Singer Jill Sobule, known for the 1995 song “I..."
4,Video shows Justin Timberlake becoming Travis ...,Timberlake posted a clip of the impromptu perf...,https://www.nbcnews.com/pop-culture/pop-cultur...,Boy band veteran Justin Timberlake brought his...
...,...,...,...,...
320,The Cold War spy mystery of the vanishing frogman,'He swam out to the Russian vessel and was nev...,https://www.bbc.com/culture/article/20250501-t...,"In 1956, Royal Navy Commander ""Buster"" Crabb d..."
321,"The long tradition of black dandyism, this yea...",'Clothes can contain a lot of emotion': How bl...,https://www.bbc.com/culture/article/20250502-h...,This year's Costume Institute spring exhibitio...
322,The true story of the first-ever 'rock star' chef,The 'first celebrity chef': How 19th-Century c...,https://www.bbc.com/culture/article/20250428-h...,A new Apple TV+ series explores the life of th...
323,Thunderbolts* is 'the greatest Marvel in years...,Thunderbolts* review: 'The greatest Marvel off...,https://www.bbc.com/culture/article/20250429-t...,"The latest in the superhero franchise ""is scra..."
